In [1]:
!pip install ptflops

In [2]:
"""
[file]          preset.py
[description]   default settings of WiFi-based models
"""
#
##
preset = {
    #
    ## define model
    "model": "resnet",                                    # "ST-RF", "MLP", "LSTM", "CNN-1D", "CNN-2D", "CLSTM", "ABLSTM", "THAT","Swin"
    #
    ## define task
    "task": "location",                                 # "identity", "activity", "location"
    #
    ## number of repeated experiments
    "repeat": 1,
    #

    #
    ## data selection for experiments
    "data": {
        "num_users": ["0", "1", "2", "3", "4", "5"],    # select number(s) of users, (e.g., ["0", "1"], ["2", "3", "4", "5"])
        "wifi_band": ["2.4"],                           # select WiFi band(s) (e.g., ["2.4"], ["5"], ["2.4", "5"])
        "environment": ["classroom"],                   # select environment(s) (e.g., ["classroom"], ["meeting_room"], ["empty_room"])
        "length": 3000,                                 # default length of CSI
    },
    #
    ## hyperparameters of models
    "nn": {
        "lr": 1e-3,                                     # learning rate
        "epoch": 120,                                   # number of epochs
        "batch_size": 32,                              # batch size
        "threshold": 0.5,                               # threshold to binarize sigmoid outputs
    },
## path of data
    "path": {
        "data_x": "/kaggle/input/wimans/wifi_csi/amp",               # directory of CSI amplitude files
        "data_y": "/kaggle/input/wimans/annotation.csv",             # path of annotation file
        "save": f"result_resnet_epoch=120_batchsize=32_envs=classroom_wifiband=2.4.json"                           # path to save results
    },
    #
    ## encoding of activities and locations
    "encoding": {
        "activity": {                                   # encoding of different activities
            "nan":      [0, 0, 0, 0, 0, 0, 0, 0, 0],
            "nothing":  [1, 0, 0, 0, 0, 0, 0, 0, 0],
            "walk":     [0, 1, 0, 0, 0, 0, 0, 0, 0],
            "rotation": [0, 0, 1, 0, 0, 0, 0, 0, 0],
            "jump":     [0, 0, 0, 1, 0, 0, 0, 0, 0],
            "wave":     [0, 0, 0, 0, 1, 0, 0, 0, 0],
            "lie_down": [0, 0, 0, 0, 0, 1, 0, 0, 0],
            "pick_up":  [0, 0, 0, 0, 0, 0, 1, 0, 0],
            "sit_down": [0, 0, 0, 0, 0, 0, 0, 1, 0],
            "stand_up": [0, 0, 0, 0, 0, 0, 0, 0, 1],
        },
        "location": {                                   # encoding of different locations
            "nan":  [0, 0, 0, 0, 0],
            "a":    [1, 0, 0, 0, 0],
            "b":    [0, 1, 0, 0, 0],
            "c":    [0, 0, 1, 0, 0],
            "d":    [0, 0, 0, 1, 0],
            "e":    [0, 0, 0, 0, 1],
        },
    },
}


In [3]:
import time
import torch
from torch import device
from torch.nn import Module
from torch.optim import Optimizer
from torch.utils.data import TensorDataset, DataLoader
from copy import deepcopy
from sklearn.metrics import accuracy_score
from tqdm import tqdm

torch.set_float32_matmul_precision("high")
torch._dynamo.config.cache_size_limit = 65536


def train(model: Module,
          optimizer: Optimizer,
          loss: Module,
          data_train_set: TensorDataset,
          data_test_set: TensorDataset,
          var_threshold: float,
          var_batch_size: int,
          var_epochs: int,
          device: device):
    data_train_loader = DataLoader(data_train_set, var_batch_size, shuffle=True, pin_memory=True)
    data_test_loader = DataLoader(data_test_set, var_batch_size)

    var_best_accuracy = 0
    var_best_weight = None

    for var_epoch in tqdm(range(var_epochs)):
        # --------------------------
        # Training
        # --------------------------
        model.train()
        total_train_loss = 0
        total_train_accuracy = 0
        train_batches = 0

        for batch_index, data_batch in enumerate(data_train_loader):

            data_batch_x, data_batch_y = data_batch
            data_batch_x = data_batch_x.to(device)
            data_batch_y = data_batch_y.to(device)

            predict_train_y = model(data_batch_x)
            var_loss_train = loss(predict_train_y, data_batch_y.reshape(data_batch_y.shape[0], -1).float())

            optimizer.zero_grad()
            var_loss_train.backward()
            optimizer.step()

            # Calculate batch accuracy
            predict_train_y = (torch.sigmoid(predict_train_y) > var_threshold).float()
            data_batch_y = data_batch_y.detach().cpu().numpy()
            predict_train_y = predict_train_y.detach().cpu().numpy()
            predict_train_y = predict_train_y.reshape(-1, data_batch_y.shape[-1])
            data_batch_y = data_batch_y.reshape(-1, data_batch_y.shape[-1])
            batch_accuracy = accuracy_score(data_batch_y.astype(float), predict_train_y.astype(float))
            total_train_loss += var_loss_train.item()
            total_train_accuracy += batch_accuracy
            train_batches += 1

            

        # Calculate average training accuracy and loss
        average_train_loss = total_train_loss / train_batches
        average_train_accuracy = total_train_accuracy / train_batches
        print(f"Epoch {var_epoch}/{var_epochs} - Average Training Loss: {average_train_loss:.6f} - "
              f"Average Training Accuracy: {average_train_accuracy:.6f}")

        # --------------------------
        # Testing
        # --------------------------
        model.eval()
        total_test_loss = 0
        total_test_accuracy = 0
        test_batches = 0

        with torch.no_grad():
            for data_batch in data_test_loader:
                data_test_x, data_test_y = data_batch
                data_test_x = data_test_x.to(device)
                data_test_y = data_test_y.to(device)

                predict_test_y = model(data_test_x)
                var_loss_test = loss(predict_test_y, data_test_y.reshape(data_test_y.shape[0], -1).float())
                predict_test_y = (torch.sigmoid(predict_test_y) > var_threshold).float()

                data_test_y = data_test_y.detach().cpu().numpy()
                predict_test_y = predict_test_y.detach().cpu().numpy()
                predict_test_y = predict_test_y.reshape(-1, data_test_y.shape[-1])
                data_test_y = data_test_y.reshape(-1, data_test_y.shape[-1])
                batch_accuracy = accuracy_score(data_test_y.astype(float), predict_test_y.astype(float))
                total_test_loss += var_loss_test.item()
                total_test_accuracy += batch_accuracy
                test_batches += 1

            
        # Calculate average testing accuracy and loss
        average_test_loss = total_test_loss / test_batches
        average_test_accuracy = total_test_accuracy / test_batches
        print(f"Epoch {var_epoch}/{var_epochs} - Average Test Loss: {average_test_loss:.6f} - "
              f"Average Test Accuracy: {average_test_accuracy:.6f}")
        print('---***---')
        # Save the best model based on average test accuracy
        if average_test_accuracy > var_best_accuracy:
            var_best_accuracy = average_test_accuracy
            var_best_weight = deepcopy(model.state_dict())
            torch.save(var_best_weight, 'best_resnet_epoch=120_batchsize=32_envs=classroom_wifiband=2.4.pth')
            print(f"New best model saved with accuracy: {var_best_accuracy:.6f}")
            print('---------------------------***---------------------------')

    return var_best_weight





In [4]:
"""
[file]          preprocess.py
[description]   preprocess WiFi CSI data
"""
#
##
import os

import numpy as np
import scipy.io as scio

#
##
def mat_to_amp(data_mat):
    """
    [description]
    : calculate amplitude of raw WiFi CSI data
    [parameter]
    : data_mat: dict, raw WiFi CSI data from *.mat files
    [return]
    : data_csi_amp: numpy array, CSI amplitude
    """
    #
    ## 
    var_length = data_mat["trace"].shape[0]
    #
    data_csi_amp = [abs(data_mat["trace"][var_t][0][0][0][-1]) for var_t in range(var_length)]
    #
    data_csi_amp = np.array(data_csi_amp, dtype = np.float32)
    #
    return data_csi_amp

#
##
def extract_csi_amp(var_dir_mat, 
                    var_dir_amp):
    """
    [description]
    : read raw WiFi CSI files (*.mat), calcuate CSI amplitude, and save amplitude (*.npy)
    [parameter]
    : var_dir_mat: string, directory to read raw WiFi CSI files (*.mat)
    : var_dir_amp: string, directory to save WiFi CSI amplitude (*.npy)
    """
    #
    ##
    var_path_mat = os.listdir(var_dir_mat)
    #
    for var_c, var_path in enumerate(var_path_mat):
        #
        data_mat = scio.loadmat(os.path.join(var_dir_mat, var_path))
        #
        data_csi_amp = mat_to_amp(data_mat)
        #
        print(var_c, data_csi_amp.shape)
        #
        var_path_save = os.path.join(var_dir_amp, var_path.replace(".mat", ".npy"))
        #
        with open(var_path_save, "wb") as var_file:
            np.save(var_file, data_csi_amp)

#
##


#


In [5]:
"""
[file]          load_data.py
[description]   load annotation file and CSI amplitude, and encode labels
"""
#
##
import os
import numpy as np
import pandas as pd
#

#
##
def load_data_y(var_path_data_y,
                var_environment = None, 
                var_wifi_band = None, 
                var_num_users = None):
    """
    [description]
    : load annotation file (*.csv) as a pandas dataframe
    : according to selected environment(s), WiFi band(s), and number(s) of users
    [parameter]
    : var_path_data_y: string, path of annotation file
    : var_environment: list, selected environment(s), e.g., ["classroom"]
    : var_wifi_band: list, selected WiFi band(s), e.g., ["2.4"]
    : var_num_users: list, selected number(s) of users, e.g., ["0", "1", "2"]
    [return]
    : data_pd_y: pandas dataframe, labels of selected data
    """
    #
    ##
    data_pd_y = pd.read_csv(var_path_data_y, dtype = str)
    #
    if var_environment is not None:
        data_pd_y = data_pd_y[data_pd_y["environment"].isin(var_environment)]
    #
    if var_wifi_band is not None:
        data_pd_y = data_pd_y[data_pd_y["wifi_band"].isin(var_wifi_band)]
    #
    if var_num_users is not None:
        data_pd_y = data_pd_y[data_pd_y["number_of_users"].isin(var_num_users)]
    #
    return data_pd_y

#
##
def load_data_x(var_path_data_x, 
                var_label_list):
    """
    [description]
    : load CSI amplitude (*.npy)
    : according to a label list of selected data
    [parameter]
    : var_path_data_x: string, directory of CSI amplitude files
    : var_label_list: list, selected labels
    [return]
    : data_x: numpy array, CSI amplitude
    """
    #
    ##
    var_path_list = [os.path.join(var_path_data_x, var_label + ".npy") for var_label in var_label_list]
    #
    data_x = []
    #
    for var_path in var_path_list:
        #
        data_csi = np.load(var_path)
        #
        var_pad_length = preset["data"]["length"] - data_csi.shape[0]
        #
        data_csi_pad = np.pad(data_csi, ((var_pad_length, 0), (0, 0), (0, 0), (0, 0)))
        #
        data_x.append(data_csi_pad)
    #
    data_x = np.array(data_x)
    #
    return data_x

#
##
def encode_data_y(data_pd_y, 
                  var_task):
    """
    [description]
    : encode labels according to specific task
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    : var_task: string, indicate task
    [return]
    : data_y: numpy array, label encoding of task
    """
    #
    ##
    if var_task == "identity":
        #
        data_y = encode_identity(data_pd_y)
    #
    elif var_task == "activity":
        #
        data_y = encode_activity(data_pd_y, preset["encoding"]["activity"])
    #
    elif var_task == "location":
        #
        data_y = encode_location(data_pd_y, preset["encoding"]["location"])
    #
    return data_y

#
##
def encode_identity(data_pd_y):
    """
    [description]
    : encode identity labels in a pandas dataframe
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    [return]
    : data_identity_onehot_y: numpy array, onehot encoding for identity labels
    """
    #
    ##
    data_location_pd_y = data_pd_y[["user_1_location", "user_2_location", 
                                    "user_3_location", "user_4_location", 
                                    "user_5_location", "user_6_location"]]
    # 
    data_identity_y = data_location_pd_y.to_numpy(copy = True).astype(str)
    #
    data_identity_y[data_identity_y != "nan"] = 1
    data_identity_y[data_identity_y == "nan"] = 0
    #
    data_identity_onehot_y = data_identity_y.astype("int8")
    #
    return data_identity_onehot_y

#
##
def encode_activity(data_pd_y, 
                    var_encoding):
    """
    [description]
    : encode activity labels in a pandas dataframe
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    : var_encoding: dict, encoding of different activities
    [return]
    : data_activity_onehot_y: numpy array, onehot encoding for activity labels
    """
    #
    ##
    data_activity_pd_y = data_pd_y[["user_1_activity", "user_2_activity", 
                                    "user_3_activity", "user_4_activity", 
                                    "user_5_activity", "user_6_activity"]]
    #
    data_activity_y = data_activity_pd_y.to_numpy(copy = True).astype(str)
    #
    data_activity_onehot_y = np.array([[var_encoding[var_y] for var_y in var_sample] for var_sample in data_activity_y])
    #
    return data_activity_onehot_y

#
##
def encode_location(data_pd_y, 
                    var_encoding):
    """
    [description]
    : encode location labels in a pandas dataframe
    [parameter]
    : data_pd_y: pandas dataframe, labels of different tasks
    : var_encoding: dict, encoding of different locations
    [return]
    : data_location_onehot_y: numpy array, onehot encoding for location labels
    """
    #
    ##
    data_location_pd_y = data_pd_y[["user_1_location", "user_2_location", 
                                    "user_3_location", "user_4_location", 
                                    "user_5_location", "user_6_location"]]
    #
    data_location_y = data_location_pd_y.to_numpy(copy = True).astype(str)
    #
    data_location_onehot_y = np.array([[var_encoding[var_y] for var_y in var_sample] for var_sample in data_location_y])
    #
    return data_location_onehot_y

#
##
def test_load_data_y():
    """
    [description]
    : test load_data_y() function
    """
    #
    ##
    print(load_data_y(preset["path"]["data_y"], 
                      var_environment = ["classroom"]).describe())
    #
    print(load_data_y(preset["path"]["data_y"], 
                      var_environment = ["meeting_room"], 
                      var_wifi_band = ["2.4"]).describe())
    #
    print(load_data_y(preset["path"]["data_y"], 
                      var_environment = ["meeting_room"], 
                      var_wifi_band = ["2.4"], 
                      var_num_users = ["1", "2", "3"]).describe())

#
##
def test_load_data_x():
    """
    [description]
    : test load_data_x() function
    """
    #
    ##
    data_pd_y = load_data_y(preset["path"]["data_y"],
                            var_environment = ["meeting_room"], 
                            var_wifi_band = ["2.4"], 
                            var_num_users = None)
    #
    var_label_list = data_pd_y["label"].to_list()
    #
    data_x = load_data_x(preset["path"]["data_x"], var_label_list)
    #
    print(data_x.shape)

#
##
def test_encode_identity():
    """
    [description]
    : test encode_identity() function
    """
    #
    ##
    data_pd_y = pd.read_csv(preset["path"]["data_y"], dtype = str)
    #
    data_identity_onehot_y = encode_identity(data_pd_y)
    #
    print(data_identity_onehot_y.shape)
    #
    print(data_identity_onehot_y[2000])

#
##
def test_encode_activity():
    """
    [description]
    : test encode_activity() function
    """
    #
    ##
    data_pd_y = pd.read_csv(preset["path"]["data_y"], dtype = str)
    #
    data_activity_onehot_y = encode_activity(data_pd_y, preset["encoding"]["activity"])
    #
    print(data_activity_onehot_y.shape)
    #
    print(data_activity_onehot_y[1560])

#
##
def test_encode_location():
    """
    [description]
    : test encode_location() function
    """
    #
    ##
    data_pd_y = pd.read_csv(preset["path"]["data_y"], dtype = str)
    #
    data_location_onehot_y = encode_location(data_pd_y, preset["encoding"]["location"])
    #
    print(data_location_onehot_y.shape)
    #
    print(data_location_onehot_y[1560])



In [6]:
"""
[file]          run.py
[description]   run WiFi-based models
"""
#
##

from torch.utils.data import DataLoader
from torchvision import models
import json


from sklearn.model_selection import train_test_split
import time
import torch
import numpy as np
#
from torch.utils.data import TensorDataset
from ptflops import get_model_complexity_info
from sklearn.metrics import classification_report, accuracy_score
#

from tqdm import tqdm

#



#
##



def run_resnet(data_train_x,
             data_train_y,
             data_test_x,
             data_test_y,
             var_repeat=10):
    """
    [description]
    : run model (ResNet18 instead of THAT) for WiFi-based classification

    [parameter]
    : data_train_x: numpy array, CSI amplitude to train model  -> shape: (N, T, F)
    : data_train_y: numpy array, labels to train model        -> shape: (N, #classes) (یا (N,) در صورت تک‌خروجی)
    : data_test_x:  numpy array, CSI amplitude to test model  -> shape: (M, T, F)
    : data_test_y:  numpy array, labels to test model         -> shape: (M, #classes)
    : var_repeat:   int, number of repeated experiments

    [return]
    : result: dict, results of experiments
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ================================ Preprocess ================================
    # reshape data for ResNet: (batch, 1, time, feature)
    data_train_x = data_train_x.reshape(data_train_x.shape[0], 1, data_train_x.shape[1], data_train_x.shape[2]*data_train_x.shape[3]*data_train_x.shape[4])
    data_test_x  = data_test_x.reshape(data_test_x.shape[0], 1, data_test_x.shape[1], data_test_x.shape[2]*data_test_x.shape[3]*data_test_x.shape[4])

    # ساخت دیتاست
    data_train_set = TensorDataset(torch.from_numpy(data_train_x).float(),
                                   torch.from_numpy(data_train_y).float())
    data_test_set  = TensorDataset(torch.from_numpy(data_test_x).float(),
                                   torch.from_numpy(data_test_y).float())

    # مشخص کردن shape ورودی و خروجی
    # در اینجا var_x_shape باید به شکل (1, time, feature) باشد (بدون درنظرگرفتن batch)
    # و var_y_shape صرفاً مشخص‌کننده‌ی ابعاد خروجی است:
    var_x_shape = data_train_x[0].shape  # (1, time, feature)
    var_y_shape = data_train_y[0].reshape(-1).shape  # (num_of_classes, )

    # ================================ تعریف مدل ResNet18 ================================
    # 1) ساخت مدل ResNet18
    model_resnet = models.resnet18(pretrained=False)

    # 2) تغییر لایه‌ی ورودی (conv1) تا بجای 3 کانال، 1 کانال دریافت کند
    model_resnet.conv1 = torch.nn.Conv2d(
        in_channels=1,       # تغییر این قسمت
        out_channels=64,
        kernel_size=7,
        stride=2,
        padding=3,
        bias=False
    )

    # 3) تغییر خروجی نهایی (fc) تا به تعداد کلاس‌های ما باشد:
    in_features_fc = model_resnet.fc.in_features  # به طور پیش‌فرض 512
    out_features_fc = var_y_shape[-1]             # مثلا تعداد کلاس‌ها
    model_resnet.fc = torch.nn.Linear(in_features_fc, out_features_fc)

    # =========================== محاسبه پارامترها و پیچیدگی (FLOPs) ===========================
    # توجه: برای ptflops باید ورودی به شکل (channels, H, W) باشد.
    # ما داریم (1, time, feature).
    var_macs, var_params = get_model_complexity_info(
        model_resnet,
        var_x_shape,  # -> (1, time, feature)
        as_strings=False
    )

    print("Parameters:", var_params, "- FLOPs:", var_macs * 2)

    # =========================== Train & Evaluate ===========================
    result = {}
    result_accuracy = []
    result_time_train = []
    result_time_test = []

    for var_r in range(var_repeat):
        print("Repeat", var_r)

        # برای تکرارپذیری
        torch.random.manual_seed(var_r + 39)

        # کپی تازه از مدل برای هر تکرار
        model_resnet_loop = models.resnet18(pretrained=False)
        model_resnet_loop.conv1 = torch.nn.Conv2d(
            in_channels=1,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        model_resnet_loop.fc = torch.nn.Linear(in_features_fc, out_features_fc)

        # انتقال به GPU/CPU
        model_resnet_loop = model_resnet_loop.to(device)

        # تعریف بهینه‌ساز
        optimizer = torch.optim.Adam(
            model_resnet_loop.parameters(),
            lr=preset["nn"]["lr"],
            weight_decay=0
        )

        # اگر مسئله‌ی شما Multi-Label باشد از BCEWithLogitsLoss استفاده می‌شود
        # اگر Multi-Class (با softmax) باشد از CrossEntropyLoss استفاده کنید.
        loss_func = torch.nn.BCEWithLogitsLoss(
            pos_weight=torch.tensor([4] * out_features_fc).to(device)
        )

        var_time_0 = time.time()

        # -------------------------------- Train --------------------------------
        var_best_weight = train(
            model=model_resnet_loop,
            optimizer=optimizer,
            loss=loss_func,
            data_train_set=data_train_set,
            data_test_set=data_test_set,
            var_threshold=preset["nn"]["threshold"],
            var_batch_size=preset["nn"]["batch_size"],
            var_epochs=preset["nn"]["epoch"],
            device=device
        )

        var_time_1 = time.time()

        # -------------------------------- Test ---------------------------------
        model_resnet_loop.load_state_dict(var_best_weight)

        # Evaluate using DataLoader
        model_resnet_loop.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            predict_test_y_list = []
            data_test_y_list = []

            for data_batch in DataLoader(data_test_set, batch_size=preset["nn"]["batch_size"], shuffle=False):
                data_test_x, data_test_y = data_batch
                data_test_x = data_test_x.to(device)
                data_test_y = data_test_y.to(device)

                predict_test_y = model_resnet_loop(data_test_x)
                predict_test_y = torch.sigmoid(predict_test_y)  # Apply sigmoid for BCEWithLogitsLoss
                predict_test_y_list.append(predict_test_y.cpu().numpy())
                data_test_y_list.append(data_test_y.cpu().numpy())

            # Concatenate results
            predict_test_y = np.vstack(predict_test_y_list)
            data_test_y = np.vstack(data_test_y_list)

            # Apply thresholding
            predict_test_y = (predict_test_y > preset["nn"]["threshold"]).astype(float)

        var_time_2 = time.time()

        # ------------------------------- Evaluate -------------------------------
        data_test_y_c = data_test_y.reshape(-1, data_test_y.shape[-1])
        predict_test_y_c = predict_test_y.reshape(-1, data_test_y.shape[-1])

        # Accuracy
        result_acc = accuracy_score(
            data_test_y_c.astype(int),
            predict_test_y_c.astype(int)
        )

        # classification report
        result_dict = classification_report(
            data_test_y_c,
            predict_test_y_c,
            digits=6,
            zero_division=0,
            output_dict=True
        )

        result[f"repeat_{var_r}"] = result_dict

        result_accuracy.append(result_acc)
        result_time_train.append(var_time_1 - var_time_0)
        result_time_test.append(var_time_2 - var_time_1)

        print("repeat_", var_r, result_accuracy)
        print(result)

    # جمع‌بندی نتایج تکرارها
    result["accuracy"] = {"avg": np.mean(result_accuracy), "std": np.std(result_accuracy)}
    result["time_train"] = {"avg": np.mean(result_time_train), "std": np.std(result_time_train)}
    result["time_test"] = {"avg": np.mean(result_time_test), "std": np.std(result_time_test)}
    result["complexity"] = {"parameter": var_params, "flops": var_macs * 2}

    return result

#
##
def run():
    """
    [description]
    : run WiFi-based models
    """
    #
    ## parse arguments from input

    #
    var_task = "location"
    var_model = "resnet"
    var_repeat = 1
    #
    ## load annotation file as labels
    data_pd_y = load_data_y(preset["path"]["data_y"],
                            var_environment = preset["data"]["environment"], 
                            var_wifi_band = preset["data"]["wifi_band"], 
                            var_num_users = preset["data"]["num_users"])
    #
    var_label_list = data_pd_y["label"].to_list()
    #
    ## load CSI amplitude
    data_x = load_data_x(preset["path"]["data_x"], var_label_list)
    #
    ## encode labels
    data_y = encode_data_y(data_pd_y, var_task)
    #
    ## a training set (80%) and a test set (20%)
    data_train_x, data_test_x, data_train_y, data_test_y = train_test_split(data_x, data_y, 
                                                                            test_size = 0.2, 
                                                                            shuffle = True, 
                                                                            random_state = 39)
    print(var_model)
    #
    ## select a WiFi-based model
    # if var_model == "ST-RF": run_model = run_strf
    #
    # if var_model == "MLP": run_model = run_mlp
    # if var_model == "swin": run_model = run_mlp
    #
    # if var_model == "LSTM": run_model = run_lstm
    # #
    # elif var_model == "CNN-1D": run_model = run_cnn_1d
    # #
    # if var_model == "CNN-2D": run_model = run_cnn_2d
    ##
    if var_model == "resnet":
        print(1)
        run_model = run_resnet
    # #
    # elif var_model == "CLSTM": run_model = run_cnn_lstm
    # #
    # elif var_model == "ABLSTM": run_model = run_ablstm
    # #
    # elif var_model == "THAT": run_model = run_that
    #
    ## run WiFi-based model
    result = run_model(data_train_x, data_train_y, 
                       data_test_x, data_test_y, var_repeat)
    #
    ##
    result["model"] = var_model
    result["task"] = var_task
    result["data"] = preset["data"]
    result["nn"] = preset["nn"]
    #
    print(result)
    #
    ## save results
    var_file = open(preset["path"]["save"], 'w')
    json.dump(result, var_file, indent = 4)

#


In [7]:
run()

resnet
1


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  11.19 M, 100.000% Params, 28.77 GMac, 99.823% MACs, 
  (conv1): Conv2d(3.14 k, 0.028% Params, 635.04 MMac, 2.203% MACs, 1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(128, 0.001% Params, 25.92 MMac, 0.090% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 12.96 MMac, 0.045% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 12.96 MMac, 0.045% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    147.97 k, 1.323% Params, 7.56 GMac, 26.228% MACs, 
    (0): BasicBlock(
      73.98 k, 0.661% Params, 3.78 GMac, 13.114% MACs, 
      (conv1): Conv2d(36.86 k, 0.330% Params, 1.88 GMac, 6.523% MACs, 64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, 0.001% Params, 6.53 MMac, 0.023% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(0, 0.000% 

  0%|          | 0/120 [00:00<?, ?it/s]

Epoch 0/120 - Average Training Loss: 0.661753 - Average Training Accuracy: 0.513520


...

 99%|█████████▉| 119/120 [1:20:06<00:40, 40.33s/it]

Epoch 118/120 - Average Test Loss: 0.681185 - Average Test Accuracy: 0.797378
---***---
Epoch 119/120 - Average Training Loss: 0.000800 - Average Training Accuracy: 0.999778


100%|██████████| 120/120 [1:20:47<00:00, 40.39s/it]

Epoch 119/120 - Average Test Loss: 0.689221 - Average Test Accuracy: 0.794219
---***---


repeat_ 0 [0.8045977011494253]
{'repeat_0': {'0': {'precision': 0.7485714285714286, 'recall': 0.6421568627450981, 'f1-score': 0.691292875989446, 'support': 204}, '1': {'precision': 0.7065217391304348, 'recall': 0.6403940886699507, 'f1-score': 0.6718346253229973, 'support': 203}, '2': {'precision': 0.7025316455696202, 'recall': 0.5967741935483871, 'f1-score': 0.6453488372093023, 'support': 186}, '3': {'precision': 0.7294117647058823, 'recall': 0.6666666666666666, 'f1-score': 0.696629213483146, 'support': 186}, '4': {'precision': 0.6914285714285714, 'recall': 0.5960591133004927, 'f1-score': 0.6402116402116402, 'support': 203}, 'micro avg': {'precision': 0.7157772621809745, 'recall': 0.6283095723014257, 'f1-score': 0.6691973969631236, 'support': 982}, 'macro avg': {'precision': 0.7156930298811874, 'recall': 0.6284101849861191, 'f1-score': 0.6690634384433064, 'support': 982}, 'weighted avg': {'precision': 0.7157168623047792, 'recall': 0.6283095723014257, 'f1-score': 0.6690194053301158, 'su